In [1]:
import os
import mne
import scipy.io
import numpy as np
import pandas as pd

In [ ]:
# List of participants (modify as needed)
participants = ['K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10']
# Define main EEG and behavioral data directories

data_dir = r'C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\EEG_data'
behav_dir = r'C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\behav_data' 

for participant in participants: 
    eeg_path = os.path.join(data_dir, participant)
    behav_path = os.path.join(behav_dir, participant)

    run_files = [f for f in os.listdir(behav_path) if f.endswith('.mat')]

    print(f"Processing {participant}...")
    print(f"EEG data path: {eeg_path}")
    print(f"Behavioral data path: {behav_path}")
    print(f"Found {len(run_files)} run files")

Processing K01...
EEG data path: C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\EEG_data\K01
Behavioral data path: C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\behav_data\K01
Found 8 run files
Processing K02...
EEG data path: C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\EEG_data\K02
Behavioral data path: C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\behav_data\K02
Found 8 run files
Processing K03...
EEG data path: C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\EEG_data\K03
Behavioral data path: C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\behav_data\K03
Found 8 run files
Processing K04...
EEG data path: C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\EEG_data\K04
Behavioral data path: C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\behav_data\K04
Found 8 run files
Processing K05...
EEG data path: C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\EEG_data\K05
Behavioral data path: C:\Users\victo\Desktop\MRes\25MResButler\250129pilo

In [14]:
ch_names = ['Fp1','Fz','F3','F7','F9','FC5','FC1','C3','T7','CP5','CP1','Pz','P3','P7','P9','O1','Oz','O2','P10','P8','P4','CP2','CP6','T8','C4','Cz','FC2','FC6','F10','F8','F4','Fp2']

occ_ch = ['Oz', 'O1', 'O2']
occ_mask = np.array([x in ['Oz','O1','O2'] for x in ch_names])

Extracting the trial orders per block

In [ ]:
image_numbers_all = {} 

for participant in participants: 
    behav_path = os.path.join(behav_dir, participant)

    run_files = [f for f in os.listdir(behav_path) if f.endswith('.mat')]

    image_numbers_participant = {}

    for i, run_file in enumerate(run_files, 1):
        print(f"Processing file: {run_file} for block {i}...")

        file_path = os.path.join(behav_path, run_file)

        #Load the .mat file
        try:
            mat_data = scipy.io.loadmat(file_path, simplify_cells=True)
            
            #Extract trial order from "design" structure
            if 'results' in mat_data and 'trial' in mat_data['results']:
                trial_data = mat_data['results']['trial']

                #convert to df
                trial_df = pd.DataFrame(trial_data)

                if 'image_nr' in trial_df: 
                    image_nr = trial_df['image_nr'].values
                    image_numbers_participant[i] = image_nr
        except Exception as e:
            print(f"Error processing {run_file} for {participant}: {e}")
            continue
    image_numbers_all[participant] = image_numbers_participant
    print(f"Processed image numbers for {participant}: {image_numbers_participant}")

# Now `image_numbers_all` contains image numbers for all participants and blocks
print("All image numbers processed:", image_numbers_all)

#Accessing data by participant: image_numbers_all['K01]
#Accessing data by participant and block: image_numbers_all['K01'][1]

Processing file: run01_eeg.mat for block 1...
Processing file: run02_eeg.mat for block 2...
Processing file: run03_eeg.mat for block 3...
Processing file: run04_eeg.mat for block 4...
Processing file: run05_eeg.mat for block 5...
Processing file: run06_eeg.mat for block 6...
Processing file: run07_eeg.mat for block 7...
Processing file: run08_eeg.mat for block 8...
Processed image numbers for K01: {1: array([45, 22, 17, 40, 51, 69, 62,  2, 14,  3, 27, 67, 18, 32, 31,  8, 66,
       37, 44,  6, 56, 67, 61, 38, 58, 42, 28, 67, 30, 54, 25, 12, 68, 24,
       20, 41, 68, 43, 57, 52,  1, 48, 66, 10, 36,  4, 23, 66, 63, 60, 47,
       59, 68, 35, 50, 15,  9, 66, 19, 49, 55, 69, 21, 11, 13, 46, 69, 33,
       16,  7, 26, 64, 66, 53, 29, 34,  5, 67, 39, 43, 37, 58, 65, 53, 15,
       54, 56, 25, 67, 33, 11, 45, 69, 27,  9, 39, 29, 68, 18, 64, 41, 62,
       69,  8, 47,  1, 69, 50, 10, 52, 17, 63, 68,  4, 38, 16, 40, 23, 67,
        7, 28, 61, 21, 68, 26,  5, 57, 69, 20, 14, 30, 31, 36, 66, 49,

In [15]:
all_data = [] 

# Loop over participants
for participant in participants:
    print(f"Processing participant: {participant}")

    # Define paths for EEG and behavioral data
    eeg_path = os.path.join(data_dir, participant)
    behav_path = os.path.join(behav_dir, participant)  # Assuming lowercase participant ID in behav data

    # Get EEG file lists
    vhdr_files = sorted([f for f in os.listdir(eeg_path) if f.endswith('.vhdr')])

    vhdr_files = vhdr_files[:8]

    # Loop over EEG runs (blocks)
    for i, vhdr_file in enumerate(vhdr_files):
        
        # Load EEG data
        file_path = os.path.join(eeg_path, vhdr_file)

        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue
        
        print(f"Processing {vhdr_file} for {participant}")

        raw = mne.io.read_raw_brainvision(file_path, preload=True)
        
        # Get events from annotations
        events, event_id = mne.events_from_annotations(raw)
        raw.pick_types(eeg=True)
        
        # Filter EEG
        raw.filter(l_freq=0.1, h_freq=40)
        
        # Select the events that match your criteria
        selected_events = np.array([x for x in events if x[2] in [4, 8]])  # Select events with ID 4 or 8
        print(f"Selected events: {selected_events}")

        # Match events with the image numbers for this block
        # Assuming you want to select events corresponding to image_nr < 65
        trueselection = [selected_events[x] for x in range(len(selected_events)) if image_nr[x] < 65]
        print(f"True selection: {trueselection}")

        # Epoching: Define event selection criteria (use trueselection as your events)
        epochs = mne.Epochs(
            raw, trueselection, tmin=-0.1, tmax=1,
            baseline=(None, 0), preload=True, picks=occ_ch
        )

        eeg_data = epochs.get_data()
        print(f"EEG data shape for {participant} block {i}: {eeg_data.shape}")
        
        #flattening the 3D array by joining timepoints and channels... not needed anymore as we have selected the spatial properties we wanted for P1! 
        eeg_data_2d = eeg_data.reshape(eeg_data.shape[0], -1)  # Flatten each epoch
        columns = [f"{ch}_{t}" for ch in occ_ch for t in range(eeg_data.shape[2])]
        
        # Create a DataFrame for this participant and block
        df = pd.DataFrame(eeg_data_2d, columns=columns)
        
        # Optionally add participant and block columns for identification
        df['participant'] = participant
        df['block'] = i + 1

        # Append the DataFrame to the list for all participants and blocks
        all_data.append(df)
        
full_data_df = pd.concat(all_data, ignore_index=True)
print("Processing complete")

Processing participant: K01
Processing k01_b1.vhdr for K01
Extracting parameters from C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\EEG_data\K01\k01_b1.vhdr...
Setting channel info structure...
Reading 0 ... 383359  =      0.000 ...   383.359 secs...


C:\Users\victo\AppData\Local\Temp\ipykernel_19660\2448625860.py:28: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(file_path, preload=True)


Used Annotations descriptions: [np.str_('New Segment/'), np.str_('Stimulus/S  1'), np.str_('Stimulus/S  4'), np.str_('Stimulus/S  8')]
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 33001 samples (33.001 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s


Selected events: [[  5255      0      4]
 [  6228      0      4]
 [  7151      0      4]
 [  8097      0      4]
 [  9068      0      4]
 [ 10068      0      8]
 [ 12041      0      4]
 [ 12959      0      4]
 [ 13908      0      4]
 [ 14849      0      4]
 [ 15836      0      4]
 [ 16789      0      8]
 [ 18713      0      4]
 [ 19707      0      4]
 [ 20657      0      4]
 [ 21592      0      4]
 [ 22595      0      8]
 [ 24582      0      4]
 [ 25496      0      4]
 [ 26468      0      4]
 [ 27415      0      4]
 [ 28389      0      8]
 [ 30390      0      4]
 [ 31364      0      4]
 [ 32369      0      4]
 [ 33339      0      4]
 [ 34309      0      4]
 [ 35251      0      8]
 [ 37161      0      4]
 [ 38153      0      4]
 [ 39084      0      4]
 [ 40031      0      4]
 [ 41004      0      8]
 [ 42965      0      4]
 [ 43952      0      4]
 [ 44859      0      4]
 [ 45841      0      8]
 [ 47835      0      4]
 [ 48739      0      4]
 [ 49650      0      4]
 [ 50619      0      4]

...

Selected events: [[  4261      0      4]
 [  5258      0      4]
 [  6185      0      4]
 [  7157      0      4]
 [  8134      0      8]
 [ 10133      0      4]
 [ 11137      0      4]
 [ 12100      0      4]
 [ 13117      0      8]
 [ 15079      0      4]
 [ 15971      0      4]
 [ 16898      0      4]
 [ 17843      0      8]
 [ 19733      0      4]
 [ 20689      0      4]
 [ 21668      0      4]
 [ 22585      0      4]
 [ 23502      0      8]
 [ 25396      0      4]
 [ 26361      0      4]
 [ 27317      0      4]
 [ 28202      0      4]
 [ 29149      0      8]
 [ 31072      0      4]
 [ 32013      0      4]
 [ 32917      0      4]
 [ 33900      0      4]
 [ 34860      0      8]
 [ 36768      0      4]
 [ 37730      0      4]
 [ 38640      0      4]
 [ 39595      0      4]
 [ 40531      0      8]
 [ 42517      0      4]
 [ 43469      0      4]
 [ 44400      0      4]
 [ 45346      0      4]
 [ 46241      0      8]
 [ 48234      0      4]
 [ 49202      0      4]
 [ 50143      0      4]

C:\Users\victo\AppData\Local\Temp\ipykernel_19660\2448625860.py:28: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(file_path, preload=True)


Used Annotations descriptions: [np.str_('New Segment/'), np.str_('Response/R  8'), np.str_('Stimulus/S  1'), np.str_('Stimulus/S  4'), np.str_('Stimulus/S  8')]
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 33001 samples (33.001 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Selected events: [[  3844      0      4]
 [  4787      0      4]
 [  5718      0      4]
 [  6603      0      4]
 [  7587      0      8]
 [  9497      0      4]
 [ 10445      0      4]
 [ 11409      0      4]
 [ 12306      0      4]
 [ 13259      0      8]
 [ 15226      0      4]
 [ 16186      0      4]
 [ 17157      0      4]
 [ 18170      0      4]
 [ 19130      0      4]
 [ 20137      0      8]
 [ 22055      0      4]
 [ 23009      0      4]
 [ 23926      0      4]
 [ 24859      0      8]
 [ 26807      0      4]
 [ 27733      0      4]
 [ 28689      0      4]
 [ 29590      0      4]
 [ 30548      0      8]
 [ 32477      0      4]
 [ 33359      0      4]
 [ 34288      0      4]
 [ 35183      0      4]
 [ 36145      0      8]
 [ 38132      0      4]
 [ 39079      0      4]
 [ 40002      0      4]
 [ 40950      0      4]
 [ 41948      0      4]
 [ 42857      0      8]
 [ 44829      0      4]
 [ 45731      0      4]
 [ 46676      0      4]
 [ 47639      0      4]
 [ 48607      0      8]

In [23]:
full_data_df.value_counts

<bound method DataFrame.value_counts of                Oz_0          Oz_1      Oz_2      Oz_3      Oz_4          Oz_5  \
0      9.658907e-06  8.793055e-06  0.000008  0.000007  0.000006  5.313380e-06   
1      6.606411e-05  6.508886e-05  0.000063  0.000061  0.000058  5.498475e-05   
2     -1.615622e-06 -2.773396e-06 -0.000004 -0.000005 -0.000006 -7.443110e-06   
3     -6.073369e-06 -5.800498e-06 -0.000005 -0.000005 -0.000004 -3.229845e-06   
4     -3.855322e-06 -4.221420e-06 -0.000004 -0.000005 -0.000004 -4.200811e-06   
...             ...           ...       ...       ...       ...           ...   
20323  9.892746e-06  9.719020e-06  0.000009  0.000008  0.000007  6.004107e-06   
20324  1.696770e-05  1.720309e-05  0.000017  0.000017  0.000016  1.466472e-05   
20325  8.104043e-07  8.949055e-07  0.000001  0.000001  0.000002  2.128299e-06   
20326  9.748110e-06  7.530335e-06  0.000005  0.000003  0.000001 -4.732387e-07   
20327  9.257857e-06  8.809772e-06  0.000008  0.000007  0.000006  4.21